<a href="https://colab.research.google.com/github/dominicwhat/Data-AI---RecSys/blob/main/sample_cf_used_based.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sample Code

## 基礎建設

In [1]:
import pandas as pd
import gzip, json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

## 載入資料

In [2]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz

--2022-01-09 13:25:14--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15499476 (15M) [application/octet-stream]
Saving to: ‘All_Beauty.csv.1’

All_Beauty.csv.1    100%[===================>]  14.78M  17.9MB/s    in 0.8s    

2022-01-09 13:25:15 (17.9 MB/s) - ‘All_Beauty.csv.1’ saved [15499476/15499476]

--2022-01-09 13:25:15--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10329961 (9.9M) [application/octet-stream]
Saving to: ‘meta_All_Beauty.json.gz.1’

meta_All_Beauty.jso 100%[===================>]   9.85M  14.2MB/s    in 0.7s   

In [3]:
metadata = getDF('/content/meta_All_Beauty.json.gz')
ratings = pd.read_csv('/content/All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)

In [4]:
metadata.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[Loud 'N Clear Personal Sound Amplifier allows...,,Loud 'N Clear&trade; Personal Sound Amplifier,[],,idea village,[],"2,938,573 in Beauty & Personal Care (",[],{'ASIN: ': '6546546450'},All Beauty,,,,6546546450,[],[]
1,[],,[No7 Lift & Luminate Triple Action Serum 50ml ...,,No7 Lift &amp; Luminate Triple Action Serum 50...,"[B01E7LCSL6, B008X5RVME]",,,[],"872,854 in Beauty & Personal Care (",[],"{'Shipping Weight:': '0.3 ounces (', 'ASIN: ':...",All Beauty,"class=""a-bordered a-horizontal-stripes a-spa...",,$44.99,7178680776,[],[]
2,[],,[No7 Stay Perfect Foundation now stays perfect...,,No7 Stay Perfect Foundation Cool Vanilla by No7,[],,No7,[],"956,696 in Beauty & Personal Care (","[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]","{'Shipping Weight:': '3.5 ounces (', 'ASIN: ':...",All Beauty,,,$28.76,7250468162,[],[]
3,[],,[],,Wella Koleston Perfect Hair Colour 44/44 Mediu...,[B0041PBXX8],,,[],"1,870,258 in Beauty & Personal Care (",[],"{' Item Weight: ': '1.76 ounces', 'Sh...",All Beauty,,,,7367905066,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,[],,[Lacto Calamine Skin Balance Daily Nourishing ...,,Lacto Calamine Skin Balance Oil control 120 ml...,[],,Pirmal Healthcare,[],"67,701 in Beauty & Personal Care (","[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4...","{'Shipping Weight:': '12 ounces (', 'ASIN: ': ...",All Beauty,,,$12.15,7414204790,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [5]:
ratings.head()

,asin,reviewerID,overall,unixReviewTime
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800
2,0143026860,A1572GUYS7DGSR,4.0,1407628800
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200


## 資料整理

In [6]:
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')

## 資料切分

In [7]:
ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01')
]
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())

## 產生推薦

In [8]:
ratings_trainings_last_sep =  ratings_trainings[
    (ratings_trainings['DATE'] >= '2017-09-01') & 
    (ratings_trainings['DATE'] <= '2017-09-30')
]
asin_group_last_sep = ratings_trainings_last_sep.groupby(by=['asin']).agg(list).reset_index()
asin_group_last_sep['interaction_num'] = [len(i) for i in asin_group_last_sep['overall']]
asin_group_last_sep = asin_group_last_sep.sort_values(by='interaction_num', ascending=False,ignore_index = True)
asin_group_last_sep.head()

,asin,reviewerID,overall,unixReviewTime,DATE,interaction_num
0,B0010ZBORW,"[A1BTG38XSGYLW3, A3S3R88HA0HZG3, A3SLC8F6VIWXI...","[5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 3.0, 5.0, 4.0, ...","[1506729600, 1506729600, 1506643200, 150664320...","[2017-09-30 00:00:00, 2017-09-30 00:00:00, 201...",400
1,B00W259T7G,"[A1RF9YK4BK5TRH, A2CNGARED6OWWU, A30Z0GG96D11E...","[3.0, 4.0, 3.0, 4.0, 5.0, 4.0, 5.0, 5.0, 5.0, ...","[1506729600, 1506729600, 1506729600, 150672960...","[2017-09-30 00:00:00, 2017-09-30 00:00:00, 201...",387
2,B01DKQAXC0,"[A1U73UEOOKPCAG, A3MTSOTN6KI5YQ, A3LHIPR8FUS9B...","[1.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, ...","[1506729600, 1506729600, 1506556800, 150655680...","[2017-09-30 00:00:00, 2017-09-30 00:00:00, 201...",124
3,B000050FDY,"[AEULHIYYGAGE2, A4QU03PP6IKZ, A3O00BUEBIFVAR, ...","[5.0, 5.0, 4.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","[1506729600, 1506729600, 1506729600, 150672960...","[2017-09-30 00:00:00, 2017-09-30 00:00:00, 201...",103
4,B001LNODUS,"[A16WPA6JV83YXT, A10ZBR6O8S8OCY, A9P07NJ7UV0M,...","[2.0, 4.0, 2.0, 5.0, 3.0, 3.0, 4.0, 5.0, 5.0, ...","[1506729600, 1506643200, 1506556800, 150647040...","[2017-09-30 00:00:00, 2017-09-29 00:00:00, 201...",87


In [9]:
import pandas as pd
from itertools import combinations
from collections import defaultdict

# header: user_id,item_id,rating,timestamp

def recommender(training_data, users=[], k=10, rule=False):

    # loading data from dataframe
    # user_to_items dict:
    # {
    #   'user': {
    #       'item': ratings...
    #   }...
    # }
    user_to_items = defaultdict(dict)
    for _, row in training_data.iterrows():
        row = dict(row)
        user = row['reviewerID']
        item = row['asin']
        rating = float(row['overall'])

        user_to_items[user][item] = rating

    print("total users before filtering: ", len(user_to_items))

    # remove obscure user to decrease data size
    # filtering params
    remove_obscure_user = True
    user_rating_threshold = 3
    all_users = list(user_to_items.keys())
    for user in all_users:
        ratings = user_to_items[user]
        if remove_obscure_user and len(ratings) < user_rating_threshold:
            del user_to_items[user]

    print("total users  after filtering: ", len(user_to_items))

    # generate item to user mapping dict
    # {
    #   'item': {
    #       'user': ratings...
    #   }...
    # }
    item_to_users = defaultdict(dict)
    for user, items in user_to_items.items():
        for item, rating in items.items():
            item_to_users[item][user] = rating

    # prepare data of computing user similarity 
    init_sim = lambda: [0 for _ in range(3)]
    factory = lambda: defaultdict(init_sim)
    pre_user_similarity = defaultdict(factory)
    n = len(item_to_users)
    index = 0
    for item, user_ratings in item_to_users.items():
        if len(user_ratings) > 1:
            # print(f"item: {item} have been rated by {len(user_ratings)} users progress: {index}/{n}")
            for user1, user2 in combinations(user_ratings.keys(), 2):
                xy = user_ratings[user1] * user_ratings[user2]
                xx = user_ratings[user1] ** 2
                yy = user_ratings[user2] ** 2
                pre_user_similarity[user1][user2][0] += xy
                pre_user_similarity[user1][user2][1] += xx
                pre_user_similarity[user1][user2][2] += yy

                pre_user_similarity[user2][user1][0] += xy
                pre_user_similarity[user2][user1][1] += xx
                pre_user_similarity[user2][user1][2] += yy
        index += 1

    user_similarity = {}
    for src_user in pre_user_similarity:
        user_similarity_order = []
        for dst_user, val in pre_user_similarity[src_user].items():
            xy = val[0]
            xx = val[1]
            yy = val[2]
            div = ((xx*yy) ** 0.5)
            if div == 0:
                continue
            similarity = xy / div
            if similarity < 0:
                continue
            for i, s in enumerate(user_similarity_order):
                target_similarity = s[1]
                if target_similarity < similarity:
                    user_similarity_order.insert(i, (dst_user, similarity))
                    break
            else:
                user_similarity_order.append((dst_user, similarity))
        user_similarity[src_user] = user_similarity_order

    recommendation = {}
    for user in users:
        if user in user_similarity:
          sim_users = user_similarity[user]
          recommended_items = []
          recommended_items_set = set()
          user_have_rated = set(user_to_items[user])
          stop_recommend = False
          for sim_user, _ in sim_users:
              items_from_sim_user = sorted(list(user_to_items[sim_user].items()), key=lambda item: item[1])
              for item, _ in items_from_sim_user:
                  if item not in user_have_rated and item not in recommended_items_set:
                      recommended_items.append(item)
                      recommended_items_set.add(item)
                  if len(recommended_items) >= k:
                      stop_recommend = True
                      break
              if stop_recommend:
                  break
          recommendation[user] = recommended_items
        else:
          if rule == True:
            recommendation[user] = asin_group_last_sep['asin'][0:k].tolist()
          else:  
            recommendation[user] = []
    return recommendation

ratings_by_user_cf = recommender(ratings_trainings, users, rule=False)
ratings_by_user_rule_cf = recommender(ratings_trainings, users, rule=True)

total users before filtering:  323489
total users  after filtering:  4793
total users before filtering:  323489
total users  after filtering:  4793


## 結果評估

In [10]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

    score = total / len(ratings_testings)
    return score

print("cf :",evaluate(ratings_testings_by_user, ratings_by_user_cf))
print("rule & cf :",evaluate(ratings_testings_by_user, ratings_by_user_rule_cf))

cf : 0.0
rule & cf : 0.1
